### Preliminary

In [5]:
# Import necessary modules
import mne
import numpy as np
import pandas as pd
import os
from mne.externals.pymatreader import read_mat

In [8]:
# Set parameters
DATA_DIR = '../data/from_osf'
EEG_DIR = 'EEG'
EXP = 1
SUBJ = 1

In [66]:
# Load EEG for one subject
mat_fname = os.path.join(DATA_DIR, 'exp{}'.format(EXP), EEG_DIR, '{}_EEG.mat'.format(SUBJ))
mat_data = read_mat(mat_fname)['eeg']
info = mne.create_info(mat_data['chanLabels'], mat_data['sampRate'])
epochs = mne.EpochsArray(
    mat_data['data'], info, tmin=mat_data['preTime'] / 1000).drop(
        mat_data['arf']['artIndCleaned'].astype(bool))
assert epochs.times[-1] == mat_data['postTime'] / 1000

Not setting metadata
Not setting metadata
960 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Dropped 151 epochs: 2, 42, 71, 78, 83, 102, 108, 111, 115, 124, 129, 134, 154, 160, 164, 175, 176, 184, 205, 207, 210, 211, 217, 218, 221, 222, 223, 224, 225, 227, 244, 246, 248, 253, 255, 302, 306, 309, 311, 317, 333, 335, 336, 338, 374, 408, 457, 469, 471, 480, 481, 485, 486, 489, 503, 504, 506, 510, 544, 545, 548, 550, 551, 557, 564, 569, 586, 590, 591, 592, 596, 614, 623, 641, 659, 660, 667, 674, 675, 677, 690, 693, 694, 695, 697, 741, 742, 749, 750, 751, 754, 757, 758, 759, 767, 789, 790, 799, 802, 803, 809, 810, 813, 814, 820, 821, 823, 824, 828, 829, 834, 841, 843, 845, 851, 853, 854, 856, 857, 858, 861, 864, 865, 866, 870, 871, 872, 873, 874, 877, 878, 879, 881, 886, 887, 890, 911, 912, 915, 916, 920, 925, 926, 930, 934, 935, 937, 938, 948, 952, 957
